### Prompt Chaining - Agentic Design Pattern

In [ ]:
import os

from llama_stack_client import LlamaStackClient, Agent, AgentEventLogger
from termcolor import colored
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [ ]:
host = os.environ["TOGETHER_URL"]
together_api_key = os.environ["TOGETHER_API_KEY"]

client = LlamaStackClient(
    base_url=host,
    provider_data={
        "tavily_search_api_key": os.getenv("TAVILY_SEARCH_API_KEY"),
        "together_api_key": together_api_key,
    },
)

In [ ]:
MODEL_ID = "meta-llama/Llama-3.3-70B-Instruct"

base_agent_config = dict(
    model=MODEL_ID,
    instructions="You are a helpful assistant.",
    sampling_params={
        "strategy": {
            "type": "top_p",
            "temperature": 1.0,
            "top_p": 0.9},
    },
)

In [ ]:
from uuid import uuid4

In [ ]:
vanilla_agent_config = {
    **base_agent_config,
    "instructions": """
    You are a helpful assistant capable of structuring data extraction and formatting. 

    You will be given tasks to extract and format data from a performance report. Here is the report:

    Q3 Performance Summary:
    Our customer satisfaction score rose to 92 points this quarter.
    Revenue grew by 45% compared to last year.
    Market share is now at 23% in our primary market.
    Customer churn decreased to 5% from 8%.
    New user acquisition cost is $43 per user.
    Product adoption rate increased to 78%.
    Employee satisfaction is at 87 points.
    Operating margin improved to 34%.
    """,
}

vanilla_agent = Agent(client, **vanilla_agent_config)
prompt_chaining_session_id = vanilla_agent.create_session(session_name=f"vanilla_agent_{uuid4()}")

In [ ]:
prompts = [
    """Extract only the numerical values and their associated metrics from the text.
    Format each as 'value: metric' on a new line.
    Example format:
    92: customer satisfaction
    45%: revenue growth""",

    """Convert all numerical values to percentages where possible.
    If not a percentage or points, convert to decimal (e.g., 92 points -> 92%).
    Keep one number per line.
    Example format:
    92%: customer satisfaction
    45%: revenue growth""",

    """Sort all lines in descending order by numerical value.
    Keep the format 'value: metric' on each line.
    Example:
    92%: customer satisfaction
    87%: employee satisfaction""",

    """Format the sorted data as a markdown table with columns:
    | Metric | Value |
    |:--|--:|
    | Customer Satisfaction | 92% |""",
]

In [ ]:
for i, prompt in enumerate(prompts):    
    response = vanilla_agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=prompt_chaining_session_id,
        stream=False,
    )
    
    print("========= Turn: ", i, "=========")
    print(response.output_message.content)
    print("\n")

In [ ]:
from rich.pretty import pprint

vanilla_agent_session = client.agents.session.retrieve(
    session_id=prompt_chaining_session_id, agent_id=vanilla_agent.agent_id)

pprint(vanilla_agent_session.to_dict())